In [1]:
import numpy as np

import empymod as epm

%load_ext line_profiler

In [2]:
nfre = [1, 100]
noff = [1, 30, 500]
nl = 5
nw = 201

print(u'   nlay   nfreq    noff    nwav        TOTAL', end='')
print(u'             NumPy          Numba         Spline        Numexpr')

nums = [0]
for nf in nfre:
    for no in noff:
        cn = nf*no*nl*nw
        if cn in nums or cn < max(nums):
            continue
        else:
            nums.append(cn)

        print(u'%7i %7i %7i %7i %12i   ::' % (nl, nf, no, nw, cn), end='')
        
        depth = np.r_[0, 1000+np.linspace(0, 5000, nl-2)]
        res = np.r_[2e14, 0.3, np.linspace(.1, 10, nl-2)]
        rec   = [np.linspace(500, min(20000, no*500+500), no)
                 , np.zeros(no), 1000]
        freq = np.linspace(0.01, 10, nf)

        model = {'src': [0, 0, 990], 'rec': rec, 'depth': depth, 'res': res,
                 'freqtime': freq, 'ab': 11, 'verb': 0}

        out1 = %timeit -q -o epm.dipole(opt=None, **model)
        out2 = %timeit -q -o epm.dipole(opt='numba', **model)
        out3 = %timeit -q -o epm.dipole(opt='spline', **model)
        out4 = %timeit -q -o epm.dipole(opt='parallel', **model)

        print(u'%10.1f ms  %10.1f ms  %10.1f ms %10.1f ms'
              % (1000*out1.best, 1000*out2.best, 1000*out3.best, 1000*out4.best))

   nlay   nfreq    noff    nwav        TOTAL             NumPy          Numba         Spline        Numexpr
      5       1       1     201         1005   ::       1.7 ms         2.4 ms         2.0 ms        2.4 ms
      5       1      30     201        30150   ::      13.3 ms        20.5 ms         2.5 ms        9.3 ms
      5       1     500     201       502500   ::     204.3 ms       194.7 ms         3.0 ms       93.0 ms
      5     100      30     201      3015000   ::    1741.3 ms      1101.4 ms       202.9 ms      618.5 ms
      5     100     500     201     50250000   ::   32461.3 ms     19419.0 ms       245.9 ms     9835.3 ms


In [3]:
nf, no, nl, nw = 100, 50, 10, 201
print(u'   nlay   nfreq    noff    nwav        TOTAL')
print(u'%7i %7i %7i %7i %12i' % (nl, nf, no, nw, cn))

%lprun -T /home/dtr/Desktop/none.txt -r -s -f epm.dipole -f epm.fem -f epm.fht -f epm.dlf -f epm.wavenumber -f epm.greenfct -f epm.reflections -f epm.fields epm.dipole(opt=None, **model)
%lprun -T /home/dtr/Desktop/numba.txt -r -s -f epm.dipole -f epm.fem -f epm.fht -f epm.dlf -f epm.wavenumber -f epm.greenfct -f epm.reflections -f epm.fields epm.dipole(opt='numba', **model)
%lprun -T /home/dtr/Desktop/spline.txt -r -s -f epm.dipole -f epm.fem -f epm.fht -f epm.dlf -f epm.wavenumber -f epm.greenfct -f epm.reflections -f epm.fields epm.dipole(opt='spline', **model)
%lprun -T /home/dtr/Desktop/parallel.txt -r -s -f epm.dipole -f epm.fem -f epm.fht -f epm.dlf -f epm.wavenumber -f epm.greenfct -f epm.reflections -f epm.fields epm.dipole(opt='parallel', **model)

   nlay   nfreq    noff    nwav        TOTAL
     10     100      50     201     50250000

*** Profile printout saved to text file '/home/dtr/Desktop/none.txt'. 

*** Profile printout saved to text file '/home/dtr/Desktop/numba.txt'. 

*** Profile printout saved to text file '/home/dtr/Desktop/spline.txt'. 

*** Profile printout saved to text file '/home/dtr/Desktop/parallel.txt'. 


In [4]:
epm.versions('HTML', ncol=3)